<a href="https://colab.research.google.com/github/ekselan/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/2_1_3_A_Copy_of_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [66]:
print(df.shape)
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


###Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [0]:
# Create copy of the dataset
data = df.copy()

In [68]:
# Subset for building class catergory
df1 = data[data['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
print(df1.shape)
df1.head()

(5061, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019


In [69]:
# Confirmation
df1['BUILDING_CLASS_CATEGORY'].value_counts()

01 ONE FAMILY DWELLINGS    5061
Name: BUILDING_CLASS_CATEGORY, dtype: int64

In [70]:
# Subset for saleprice
df1 = df1[df1['SALE_PRICE'] > 100_000]
print(df1.shape)
df1.head()

(3232, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [71]:
df1 = df1[df1['SALE_PRICE'] < 2_000_000]
print(df1.shape)
df1.head()

(3151, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [72]:
df1.sort_values('SALE_PRICE')
# Subset now includes data with sale price > 100_000 and < 2_000_000

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18147,4,OTHER,01 ONE FAMILY DWELLINGS,1,11159,11,NaN,A0,223-29 103RD AVENUE,NaN,11429.0,1.0,0.0,1.0,"3,910",1163.0,1950.0,1,A0,104000,03/30/2019
3489,4,OTHER,01 ONE FAMILY DWELLINGS,1,16118,56,NaN,A5,313 BEACH 86 STREET,NaN,11693.0,1.0,0.0,1.0,"1,802",1205.0,1920.0,1,A5,105000,01/18/2019
19961,5,OTHER,01 ONE FAMILY DWELLINGS,1,1087,14,NaN,A1,147 HARRISON AVENUE,NaN,10302.0,1.0,0.0,1.0,"2,525",1336.0,1899.0,1,A1,107500,04/09/2019
791,2,OTHER,01 ONE FAMILY DWELLINGS,1,3992,55,NaN,A2,2425 ST. RAYMONDS AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,590",1400.0,1925.0,1,A2,108000,01/07/2019
15768,2,OTHER,01 ONE FAMILY DWELLINGS,1D,5514,62,NaN,A8,"81D EDGEWATER PARK, 81D",NaN,10465.0,0.0,0.0,0.0,0,0.0,1930.0,1,A8,110000,03/20/2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19698,4,OTHER,01 ONE FAMILY DWELLINGS,1,4509,54,NaN,A1,150-57 6TH AVENUE,NaN,11357.0,1.0,0.0,1.0,"5,000",2432.0,2016.0,1,A1,1909219,04/08/2019
22112,3,OTHER,01 ONE FAMILY DWELLINGS,1,5141,88,NaN,A1,296 STRATFORD ROAD,NaN,11218.0,1.0,0.0,1.0,"6,750",2275.0,1920.0,1,A1,1912500,04/23/2019
7115,3,OTHER,01 ONE FAMILY DWELLINGS,1,5050,2,NaN,S0,625 ROGERS AVENUE,NaN,11225.0,1.0,2.0,3.0,"3,700",2128.0,1905.0,1,S0,1925000,02/05/2019
7554,3,OTHER,01 ONE FAMILY DWELLINGS,1,6101,11,NaN,A1,1 MARINE AVENUE,NaN,11209.0,1.0,0.0,1.0,"4,290",1960.0,1925.0,1,A1,1950000,02/07/2019


###Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
df1['SALE_DATE'] = pd.to_datetime(df1['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = df1[df1['SALE_DATE'] < cutoff]
test = df1[df1['SALE_DATE'] >= cutoff]

In [74]:
train['SALE_DATE'].value_counts()

2019-01-31    78
2019-03-29    62
2019-02-28    58
2019-01-15    57
2019-01-24    56
              ..
2019-01-01     2
2019-03-30     1
2019-03-17     1
2019-03-09     1
2019-02-17     1
Name: SALE_DATE, Length: 68, dtype: int64

In [75]:
test['SALE_DATE'].value_counts()
# Train and test data now split

2019-04-15    43
2019-04-02    41
2019-04-01    41
2019-04-10    41
2019-04-04    40
2019-04-09    39
2019-04-12    35
2019-04-18    35
2019-04-03    33
2019-04-05    33
2019-04-11    32
2019-04-16    31
2019-04-17    28
2019-04-30    26
2019-04-08    26
2019-04-22    21
2019-04-29    21
2019-04-25    18
2019-04-23    18
2019-04-19    18
2019-04-24    15
2019-04-26     8
2019-04-14     1
Name: SALE_DATE, dtype: int64

In [76]:
print(train.shape)
print(test.shape)

(2507, 21)
(644, 21)


###Do one-hot encoding of categorical features.

In [77]:
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,1,2507,2507,2507
unique,5,6,1,2,13,2497,1,887,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,117-45 125TH STREET,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1204,2382,2507,2476,919,2,1,234,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [78]:
# Looks like a few of these have high-cardinality, so I'll exclude them

target = 'SALE_PRICE'
high_cardinal = ['ADDRESS','LAND_SQUARE_FEET','SALE_DATE']
features = train.columns.drop([target] + high_cardinal)

features

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'TAX_CLASS_AT_TIME_OF_SALE',
       'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [79]:
# Specify x and y train data
X_train = train[features]
y_train = train[target]
print(X_train.shape)
# Specify x and y test data
X_test = test[features]
y_test = test[target]

(2507, 17)


In [0]:
# Now I'll encode the non-numeric features

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [81]:
print(X_train.shape)
print(X_test.shape)
X_train.head()
# Encoded columns are now added

(2507, 50)
(644, 50)


,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,APARTMENT_NUMBER_nan,APARTMENT_NUMBER_RP.,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5495,801,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,1,0,0,0,0,0,1,1,0,7918,72,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,1,1,0,4210,19,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,1,1,0,5212,69,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,1,1,0,7930,121,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,0,0,1,0,0,0,0,0,0,0,0


###Do feature selection with SelectKBest.

In [82]:
X_train.isnull().sum()
# Looks like EASE-MENT column is entirely NaN, so I'll remove

BOROUGH_3                                             0
BOROUGH_4                                             0
BOROUGH_2                                             0
BOROUGH_5                                             0
BOROUGH_1                                             0
NEIGHBORHOOD_OTHER                                    0
NEIGHBORHOOD_FLUSHING-NORTH                           0
NEIGHBORHOOD_BEDFORD STUYVESANT                       0
NEIGHBORHOOD_FOREST HILLS                             0
NEIGHBORHOOD_BOROUGH PARK                             0
NEIGHBORHOOD_ASTORIA                                  0
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS       0
TAX_CLASS_AT_PRESENT_1                                0
TAX_CLASS_AT_PRESENT_1D                               0
BLOCK                                                 0
LOT                                                   0
EASE-MENT                                          2507
BUILDING_CLASS_AT_PRESENT_A9                    

In [83]:
X_train = X_train.drop('EASE-MENT', axis=1)
X_test = X_test.drop('EASE-MENT', axis=1)
print(X_train.shape)
print(X_test.shape)
# NaN values are now removed

(2507, 49)
(644, 49)


In [84]:
# Using f_regression as the score function in prep for ridge regression
selector = SelectKBest(score_func=f_regression, k=20)

# Apply to data
X_train_s = selector.fit_transform(X_train, y_train)
X_test_s = selector.transform(X_test)

print(X_train_s.shape)
print(X_test_s.shape)

(2507, 20)
(644, 20)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [85]:
# Review which features were selected

s_mask = selector.get_support()
all_names = X_train.columns
s_names = all_names[s_mask]
u_names = all_names[~s_mask]

print('Features selected:')
for name in s_names:
    print(name)

print('\n')
print('Features not selected:')
for name in u_names:
    print(name)

Features selected:
BOROUGH_3
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
BLOCK
BUILDING_CLASS_AT_PRESENT_A5
BUILDING_CLASS_AT_PRESENT_A3
BUILDING_CLASS_AT_PRESENT_S1
BUILDING_CLASS_AT_PRESENT_A6
ZIP_CODE
COMMERCIAL_UNITS
TOTAL_UNITS
GROSS_SQUARE_FEET
BUILDING_CLASS_AT_TIME_OF_SALE_A5
BUILDING_CLASS_AT_TIME_OF_SALE_A3
BUILDING_CLASS_AT_TIME_OF_SALE_S1
BUILDING_CLASS_AT_TIME_OF_SALE_A6


Features not selected:
BOROUGH_4
BOROUGH_1
NEIGHBORHOOD_BEDFORD STUYVESANT
NEIGHBORHOOD_ASTORIA
BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
LOT
BUILDING_CLASS_AT_PRESENT_A9
BUILDING_CLASS_AT_PRESENT_A1
BUILDING_CLASS_AT_PRESENT_A0
BUILDING_CLASS_AT_PRESENT_A2
BUILDING_CLASS_AT_PRESENT_A4
BUILDING_CLASS_AT_PRESENT_A8
BUILDING_CLASS_AT_PRESENT_B2
BUILDING_CLASS_AT_PRESENT_S0
BUILDING_CLASS_AT_PRESENT_B3
APARTMENT_NUMBER_nan
APARTMENT_NUMBER_RP.
RESIDENTIAL_UNITS
YEAR_BUILT
TAX_CLASS

In [55]:
train.corr()
# Out of curiosity, wanted to look at correlations from train data ...
# zip code and gross square feet are main features I'd like to keep, 
# both of which were selected by SelectKBest

# Block, commercial units and total units also looked relevant, so
# I think I'm satisfied with this result

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
BLOCK,1.000000,0.048937,NaN,0.644818,-0.171459,-0.027522,-0.131137,-0.134267,-0.189535,NaN,-0.147684
LOT,0.048937,1.000000,NaN,-0.037589,-0.104133,-0.030540,-0.090103,-0.021153,0.050551,NaN,-0.047241
EASE-MENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,0.644818,-0.037589,NaN,1.000000,-0.069325,0.045771,-0.009976,-0.073119,-0.434665,NaN,0.205550
RESIDENTIAL_UNITS,-0.171459,-0.104133,NaN,-0.069325,1.000000,0.014083,0.654053,0.275252,0.048933,NaN,0.087763
COMMERCIAL_UNITS,-0.027522,-0.030540,NaN,0.045771,0.014083,1.000000,0.765585,0.155831,-0.092185,NaN,0.123074
TOTAL_UNITS,-0.131137,-0.090103,NaN,-0.009976,0.654053,0.765585,1.000000,0.294987,-0.038256,NaN,0.149575
GROSS_SQUARE_FEET,-0.134267,-0.021153,NaN,-0.073119,0.275252,0.155831,0.294987,1.000000,0.130416,NaN,0.454418
YEAR_BUILT,-0.189535,0.050551,NaN,-0.434665,0.048933,-0.092185,-0.038256,0.130416,1.000000,NaN,-0.037376
TAX_CLASS_AT_TIME_OF_SALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [0]:
# Choose features
features = ['BOROUGH_3','BOROUGH_2','BOROUGH_5','NEIGHBORHOOD_OTHER',
            'NEIGHBORHOOD_FLUSHING-NORTH','NEIGHBORHOOD_FOREST HILLS',
            'NEIGHBORHOOD_BOROUGH PARK','BLOCK','BUILDING_CLASS_AT_PRESENT_A5',
            'BUILDING_CLASS_AT_PRESENT_A3','BUILDING_CLASS_AT_PRESENT_S1',
            'BUILDING_CLASS_AT_PRESENT_A6','ZIP_CODE','COMMERCIAL_UNITS',
            'TOTAL_UNITS','GROSS_SQUARE_FEET','BUILDING_CLASS_AT_TIME_OF_SALE_A5',
            'BUILDING_CLASS_AT_TIME_OF_SALE_A3','BUILDING_CLASS_AT_TIME_OF_SALE_S1',
            'BUILDING_CLASS_AT_TIME_OF_SALE_A6']

# Instantiate the model
ridge = Ridge(alpha=.001, normalize=True)

# Fit the model with all selected features
ridge.fit(X_train[features], y_train)

# Establish predictions
y_pred = ridge.predict(X_test_s)

###Get mean absolute error for the test set.

In [101]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean absolue error (test set): ${mae:.0f}')

Mean absolue error (test set): $159165


# Stretch - try linear regression

In [0]:
model = LinearRegression()
model.fit(X_train[features], y_train)
y_pred = model.predict(X_test_s)

In [98]:
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean absolue error (test set - Linear): ${mae:.0f}')

# Looks like linear regression was about the same

Mean absolue error (test set - Linear): $159303


# Stretch - try RidgeCV

In [99]:
ridge = RidgeCV(alphas = (0.001, 0.01, 0.1, 1.0, 10, 100), normalize=True)
ridge.fit(X_train[features], y_train)
ridge.alpha_

0.001